# Билет 36

# Задание 1

# Задание 2

<img src="img/36_2.png" width=800>

In [46]:
import xlwings as xw

In [47]:
wb = xw.Book('data/task_2/себестоимостьА_в1.xlsx')

In [48]:
recipe_sheet: xw.Sheet = wb.sheets['Рецептура']
price_sheet: xw.Sheet = wb.sheets.add('Цена ресурсов')

In [49]:
def get_resources_prices(sheet: xw.Sheet, i: int, j: int) -> list[tuple[str, float]]:
    resources = sheet.range(i + 2, j + 5).expand('right').value
    height = sheet.range(i + 2 + 1, j).expand('down').shape[0]
    prices = sheet.range(i + height, j + 5).expand('right').value
    return list(zip(resources, prices))


def get_next_pos(sheet: xw.Sheet, i: int, j: int) -> tuple[int, int]:
    i = i + 2 + 1
    height = sheet.range(i, j).expand('down').shape[0]
    i = i + height + 2
    if not sheet.range(i, j).value:
        raise StopIteration
    return i, j


def replace_by_formula(sheet: xw.Sheet, i: int, j: int):
    formula = f"=VLOOKUP(G{i + 2},'{price_sheet.name}'!$A$2:$B${len(resources_prices) + 1},2,0)"
    height = sheet.range(i + 2 + 1, j).expand('down').shape[0]
    sheet.range(i + height, j + 5).expand('right').formula = formula

In [50]:
i, j = 3, 2
resources_prices = {}
while True:
    try:
        for resource, price in get_resources_prices(recipe_sheet, i, j):
            resources_prices[resource] = price
        i, j = get_next_pos(recipe_sheet, i, j)
    except StopIteration:
        break

print(resources_prices)

{'Вода': 7.0, 'Пшеничная мука': 10.0, 'Дрожжи': 184.0, 'Сахар': 19.0, 'Соль': 7.0, 'Куркума молотая': 290.0, 'Топленное масло': 105.0, 'Кунжутные семена': 83.0, 'Масло оливковое': 260.0, 'Ржаная мука': 10.0, 'Солод ржаной': 108.0, 'Цельнозерновая мука': 12.0, 'Гречневая мука': 12.0, 'Ячменная мука': 12.0, 'Кукурузная мука': 12.0, 'Молоко': 40.0, 'Чеснок ': 1.0, 'Овсянные хлопья': 18.0, 'Пахта': 40.0, 'Изюм': 83.0, 'Семена подсолнуха': 160.0, 'Масло сливочное': 150.0, 'Яйцо': 4.0, 'Джем': 63.0, 'Сыр': 110.0, 'Ветчина': 270.0, 'Лосось': 250.0, 'Зелень': 280.0, 'Шоколад': 200.0, 'Творог': 80.0, 'Яблоки': 25.0, 'Вишня': 93.0}


In [51]:
price_sheet.range('A1').value = [('Ресурс', 'Цена'), *resources_prices.items()]

In [52]:
i, j = 3, 2
while True:
    try:
        replace_by_formula(recipe_sheet, i, j)
        i, j = get_next_pos(recipe_sheet, i, j)
    except StopIteration:
        break

In [11]:
# wb.save()
wb.close()

# Задание 3

<img src="img/36_3.png" width=800>

In [95]:
import re
from operator import add
import dask.bag as db

In [115]:
pattern = re.compile(r'^\s*[-—].+')


def count_direct_speech(prev: int, line: str) -> int:
    return prev + 1 if pattern.match(line) else prev

In [116]:
bag = db.read_text('data/task_3/all_k/*', encoding='cp1251')
bag = bag.fold(count_direct_speech, combine=add, initial=0)
bag.compute()

898765